<a href="https://colab.research.google.com/github/Seeker220/Mover/blob/main/Mover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install RCLONE

In [ ]:
!sudo -v ; curl https://rclone.org/install.sh | sudo bash

# Initialize Config File

In [2]:
!rm -f /root/.config/rclone/rclone.conf
!mkdir -p /root/.config/rclone/ && touch /root/.config/rclone/rclone.conf

# Mount Drive 1

In [ ]:
import subprocess, time, json, os
import requests
import urllib.parse
from datetime import datetime, timedelta, timezone

# Input Parameters
drive_name = "Drive1"
drive_type = "GDrive"  # @param ["OneDrive", "GDrive", "Google Shared Drive"]

def formatx(i, j, k):
    val = auth_response[i].split(": ")[1][1-j:-2+j+k]
    return val

# Constants for OneDrive
one_drive_client_id = "b15665d9-eda6-4092-8539-0eec376afd59"
one_drive_client_secret = "qtyfaBBYA403=unZUP40~_#"
redirect_uri = "http://localhost:53682/"
auth_endpoint = "https://login.microsoftonline.com/common/oauth2/v2.0/authorize"
token_endpoint = "https://login.microsoftonline.com/common/oauth2/v2.0/token"
graph_api_endpoint = "https://graph.microsoft.com/v1.0"
rclone_config_path = "/root/.config/rclone/rclone.conf"

if drive_type == "OneDrive":
    # Step 1: Generate the authorization URL
    params = {
        "client_id": one_drive_client_id,
        "response_type": "code",
        "redirect_uri": redirect_uri,
        "scope": "Files.ReadWrite offline_access",
        "response_mode": "query"
    }
    auth_url = f"{auth_endpoint}?{urllib.parse.urlencode(params)}"

    print("Open the following URL in your browser to authorize the application:")
    print(auth_url)

    # Step 2: Wait for the user to input the authorization code URL
    redirect_url = input("Enter the full redirect URL after authorization: ")

    # Step 3: Extract the authorization code from the URL
    auth_code = urllib.parse.parse_qs(urllib.parse.urlparse(redirect_url).query).get('code', [None])[0]

    if not auth_code:
        print("Failed to extract authorization code from the URL.")
        exit(1)

    # Step 4: Exchange the authorization code for an access token and refresh token
    data = {
        "client_id": one_drive_client_id,
        "client_secret": one_drive_client_secret,
        "grant_type": "authorization_code",
        "code": auth_code,
        "redirect_uri": redirect_uri
    }

    response = requests.post(token_endpoint, data=data)
    if response.status_code == 200:
        tokens = response.json()
        access_token = tokens.get("access_token")
        refresh_token = tokens.get("refresh_token")
        token_type = tokens.get("token_type")
        expires_in = tokens.get("expires_in")

        # Calculate the expiry date
        expiry_date = datetime.now(timezone.utc) + timedelta(seconds=expires_in)
        expiry_date_str = expiry_date.isoformat()

        print("Access Token:", access_token)
        print("Refresh Token:", refresh_token)
        print("Token Type:", token_type)
        print("Expires In:", expires_in)
        print("Expiry Date:", expiry_date_str)

        # Step 5: Use the Microsoft Graph API to find the drives
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        response = requests.get(f"{graph_api_endpoint}/me/drives", headers=headers)
        if response.status_code == 200:
            drives = response.json().get('value', [])
            if drives:
                print("Available Drives:")
                for i, drive in enumerate(drives):
                    print(f"{i + 1}. {drive.get('name')} ({drive.get('driveType')})")
                    print(f"   \\ ({drive.get('id')})")

                # Ask the user to select a drive
                selected_index = input("Enter the number of the drive you want to use (or type your own drive ID): ")
                if selected_index.isdigit() and 1 <= int(selected_index) <= len(drives):
                    selected_drive = drives[int(selected_index) - 1]
                    drive_id = selected_drive.get("id")
                    drive_type = selected_drive.get("driveType")
                else:
                    drive_id = selected_index
                    drive_type = "business" if "&session_state=" in redirect_url else "personal"

                print("Selected Drive ID:", drive_id)
                print("Selected Drive Type:", drive_type)

                # Generate the configuration text
                config_text = f"""
[{drive_name}]
type = onedrive
token = {{"access_token":"{access_token}","token_type":"{token_type}","refresh_token":"{refresh_token}","expiry":"{expiry_date_str}"}}
drive_id = {drive_id}
drive_type = {drive_type}
"""
                # Append the configuration to the rclone.conf file
                with open(rclone_config_path, "a") as config_file:
                    config_file.write("\n" + config_text)

                print("\nConfiguration has been appended to rclone.conf.")
            else:
                print("No drives found.")
        else:
            print("Failed to retrieve drives.")
            print("Status Code:", response.status_code)
            print("Response:", response.json())
    else:
        print("Failed to retrieve tokens")
        print("Status Code:", response.status_code)
        print("Response:", response.json())

else:
    # GDrive or Google Shared Drive
    client_id = "202264815644.apps.googleusercontent.com"
    client_secret = "X4Z3ca8xfWDb1Voo-F9a7ZxJ"
    redirect_uri = "http://localhost:53682/"
    auth_endpoint = "https://accounts.google.com/o/oauth2/auth"
    token_endpoint = "https://oauth2.googleapis.com/token"

    params = {
        "client_id": client_id,
        "response_type": "code",
        "redirect_uri": redirect_uri,
        "scope": "https://www.googleapis.com/auth/drive",
        "access_type": "offline",
        "prompt": "consent"
    }
    auth_url = f"{auth_endpoint}?{urllib.parse.urlencode(params)}"

    print("Open the following URL in your browser to authorize the application:")
    print(auth_url)

    # Step 2: Wait for the user to input the authorization code URL
    redirect_url = input("Enter the full redirect URL after authorization: ")

    # Step 3: Extract the authorization code from the URL
    auth_code = urllib.parse.parse_qs(urllib.parse.urlparse(redirect_url).query).get('code', [None])[0]

    if not auth_code:
        print("Failed to extract authorization code from the URL.")
        exit(1)

    # Step 4: Exchange the authorization code for an access token and refresh token
    data = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "authorization_code",
        "code": auth_code,
        "redirect_uri": redirect_uri
    }

    response = requests.post(token_endpoint, data=data)
    if response.status_code == 200:
        tokens = response.json()
        access_token = tokens.get("access_token")
        refresh_token = tokens.get("refresh_token")
        token_type = tokens.get("token_type")
        expires_in = tokens.get("expires_in")

        # Calculate the expiry date
        expiry_date = datetime.now(timezone.utc) + timedelta(seconds=expires_in)
        expiry_date_str = expiry_date.isoformat()

        print("Access Token:", access_token)
        print("Refresh Token:", refresh_token)
        print("Token Type:", token_type)
        print("Expires In:", expires_in)
        print("Expiry Date:", expiry_date_str)

        # Fetch the root ID or team drive ID based on the selected drive type
        if drive_type == "Google Shared Drive":
            print("Fetching team drive list...")
            check_drives = subprocess.check_output(f'''
            curl 'https://www.googleapis.com/drive/v3/drives' \
            --header 'Authorization: Bearer {access_token}' \
            --header 'Accept: application/json' \
            --compressed''',  shell=True).decode()
            drive_list = json.loads(check_drives).get('drives')
            print("Choose a number from below. (If you want to use the 1st team drive from the below list, type 1 and press enter)")
            for i in range(len(drive_list)):
                print(i+1, "/", drive_list[i].get('name'))
                print("  \\", "\""+drive_list[i].get('id')+"\"")
            selected_num = int(input("Enter the number: "))
            root_id = drive_list[selected_num-1].get('id')
        else:
            print("Fetching root id...")
            check_about = subprocess.check_output(f'''
            curl 'https://www.googleapis.com/drive/v2/about' \
            --header 'Authorization: Bearer {access_token}' \
            --header 'Accept: application/json' \
            --compressed''',  shell=True).decode()
            root_id = json.loads(check_about).get('rootFolderId')

        # Read file
        rfile = open(r"/root/.config/rclone/rclone.conf", "r")
        rfile_content = rfile.readlines()
        rfile.close()

        # Write file
        wfile = open(r"/root/.config/rclone/rclone.conf", "a+")
        if len(rfile_content) > 0:
            wfile.write("\n\n")
        wfile.write("[{}]\ntype = drive\nscope = drive\ntoken = {}\n".format(drive_name, json.dumps(tokens)))
        if drive_type == "Google Shared Drive":
            wfile.write("team_drive = {}".format(root_id))
        else:
            wfile.write("root_folder_id = {}".format(root_id))
        wfile.close()

        print("\nConfiguration has been appended to rclone.conf.")
    else:
        print("Failed to retrieve tokens")
        print("Status Code:", response.status_code)
        print("Response:", response.json())

# Mount Drive 2

In [ ]:
import subprocess, time, json, os
import requests
import urllib.parse
from datetime import datetime, timedelta, timezone

# Input Parameters
drive_name = "Drive2"
drive_type = "OneDrive"  # @param ["OneDrive", "GDrive", "Google Shared Drive"]

def formatx(i, j, k):
    val = auth_response[i].split(": ")[1][1-j:-2+j+k]
    return val

# Constants for OneDrive
one_drive_client_id = "b15665d9-eda6-4092-8539-0eec376afd59"
one_drive_client_secret = "qtyfaBBYA403=unZUP40~_#"
redirect_uri = "http://localhost:53682/"
auth_endpoint = "https://login.microsoftonline.com/common/oauth2/v2.0/authorize"
token_endpoint = "https://login.microsoftonline.com/common/oauth2/v2.0/token"
graph_api_endpoint = "https://graph.microsoft.com/v1.0"
rclone_config_path = "/root/.config/rclone/rclone.conf"

if drive_type == "OneDrive":
    # Step 1: Generate the authorization URL
    params = {
        "client_id": one_drive_client_id,
        "response_type": "code",
        "redirect_uri": redirect_uri,
        "scope": "Files.ReadWrite offline_access",
        "response_mode": "query"
    }
    auth_url = f"{auth_endpoint}?{urllib.parse.urlencode(params)}"

    print("Open the following URL in your browser to authorize the application:")
    print(auth_url)

    # Step 2: Wait for the user to input the authorization code URL
    redirect_url = input("Enter the full redirect URL after authorization: ")

    # Step 3: Extract the authorization code from the URL
    auth_code = urllib.parse.parse_qs(urllib.parse.urlparse(redirect_url).query).get('code', [None])[0]

    if not auth_code:
        print("Failed to extract authorization code from the URL.")
        exit(1)

    # Step 4: Exchange the authorization code for an access token and refresh token
    data = {
        "client_id": one_drive_client_id,
        "client_secret": one_drive_client_secret,
        "grant_type": "authorization_code",
        "code": auth_code,
        "redirect_uri": redirect_uri
    }

    response = requests.post(token_endpoint, data=data)
    if response.status_code == 200:
        tokens = response.json()
        access_token = tokens.get("access_token")
        refresh_token = tokens.get("refresh_token")
        token_type = tokens.get("token_type")
        expires_in = tokens.get("expires_in")

        # Calculate the expiry date
        expiry_date = datetime.now(timezone.utc) + timedelta(seconds=expires_in)
        expiry_date_str = expiry_date.isoformat()

        print("Access Token:", access_token)
        print("Refresh Token:", refresh_token)
        print("Token Type:", token_type)
        print("Expires In:", expires_in)
        print("Expiry Date:", expiry_date_str)

        # Step 5: Use the Microsoft Graph API to find the drives
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        response = requests.get(f"{graph_api_endpoint}/me/drives", headers=headers)
        if response.status_code == 200:
            drives = response.json().get('value', [])
            if drives:
                print("Available Drives:")
                for i, drive in enumerate(drives):
                    print(f"{i + 1}. {drive.get('name')} ({drive.get('driveType')})")
                    print(f"   \\ ({drive.get('id')})")

                # Ask the user to select a drive
                selected_index = input("Enter the number of the drive you want to use (or type your own drive ID): ")
                if selected_index.isdigit() and 1 <= int(selected_index) <= len(drives):
                    selected_drive = drives[int(selected_index) - 1]
                    drive_id = selected_drive.get("id")
                    drive_type = selected_drive.get("driveType")
                else:
                    drive_id = selected_index
                    drive_type = "business" if "&session_state=" in redirect_url else "personal"

                print("Selected Drive ID:", drive_id)
                print("Selected Drive Type:", drive_type)

                # Generate the configuration text
                config_text = f"""
[{drive_name}]
type = onedrive
token = {{"access_token":"{access_token}","token_type":"{token_type}","refresh_token":"{refresh_token}","expiry":"{expiry_date_str}"}}
drive_id = {drive_id}
drive_type = {drive_type}
"""
                # Append the configuration to the rclone.conf file
                with open(rclone_config_path, "a") as config_file:
                    config_file.write("\n" + config_text)

                print("\nConfiguration has been appended to rclone.conf.")
            else:
                print("No drives found.")
        else:
            print("Failed to retrieve drives.")
            print("Status Code:", response.status_code)
            print("Response:", response.json())
    else:
        print("Failed to retrieve tokens")
        print("Status Code:", response.status_code)
        print("Response:", response.json())

else:
    # GDrive or Google Shared Drive
    client_id = "202264815644.apps.googleusercontent.com"
    client_secret = "X4Z3ca8xfWDb1Voo-F9a7ZxJ"
    redirect_uri = "http://localhost:53682/"
    auth_endpoint = "https://accounts.google.com/o/oauth2/auth"
    token_endpoint = "https://oauth2.googleapis.com/token"

    params = {
        "client_id": client_id,
        "response_type": "code",
        "redirect_uri": redirect_uri,
        "scope": "https://www.googleapis.com/auth/drive",
        "access_type": "offline",
        "prompt": "consent"
    }
    auth_url = f"{auth_endpoint}?{urllib.parse.urlencode(params)}"

    print("Open the following URL in your browser to authorize the application:")
    print(auth_url)

    # Step 2: Wait for the user to input the authorization code URL
    redirect_url = input("Enter the full redirect URL after authorization: ")

    # Step 3: Extract the authorization code from the URL
    auth_code = urllib.parse.parse_qs(urllib.parse.urlparse(redirect_url).query).get('code', [None])[0]

    if not auth_code:
        print("Failed to extract authorization code from the URL.")
        exit(1)

    # Step 4: Exchange the authorization code for an access token and refresh token
    data = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "authorization_code",
        "code": auth_code,
        "redirect_uri": redirect_uri
    }

    response = requests.post(token_endpoint, data=data)
    if response.status_code == 200:
        tokens = response.json()
        access_token = tokens.get("access_token")
        refresh_token = tokens.get("refresh_token")
        token_type = tokens.get("token_type")
        expires_in = tokens.get("expires_in")

        # Calculate the expiry date
        expiry_date = datetime.now(timezone.utc) + timedelta(seconds=expires_in)
        expiry_date_str = expiry_date.isoformat()

        print("Access Token:", access_token)
        print("Refresh Token:", refresh_token)
        print("Token Type:", token_type)
        print("Expires In:", expires_in)
        print("Expiry Date:", expiry_date_str)

        # Fetch the root ID or team drive ID based on the selected drive type
        if drive_type == "Google Shared Drive":
            print("Fetching team drive list...")
            check_drives = subprocess.check_output(f'''
            curl 'https://www.googleapis.com/drive/v3/drives' \
            --header 'Authorization: Bearer {access_token}' \
            --header 'Accept: application/json' \
            --compressed''',  shell=True).decode()
            drive_list = json.loads(check_drives).get('drives')
            print("Choose a number from below. (If you want to use the 1st team drive from the below list, type 1 and press enter)")
            for i in range(len(drive_list)):
                print(i+1, "/", drive_list[i].get('name'))
                print("  \\", "\""+drive_list[i].get('id')+"\"")
            selected_num = int(input("Enter the number: "))
            root_id = drive_list[selected_num-1].get('id')
        else:
            print("Fetching root id...")
            check_about = subprocess.check_output(f'''
            curl 'https://www.googleapis.com/drive/v2/about' \
            --header 'Authorization: Bearer {access_token}' \
            --header 'Accept: application/json' \
            --compressed''',  shell=True).decode()
            root_id = json.loads(check_about).get('rootFolderId')

        # Read file
        rfile = open(r"/root/.config/rclone/rclone.conf", "r")
        rfile_content = rfile.readlines()
        rfile.close()

        # Write file
        wfile = open(r"/root/.config/rclone/rclone.conf", "a+")
        if len(rfile_content) > 0:
            wfile.write("\n\n")
        wfile.write("[{}]\ntype = drive\nscope = drive\ntoken = {}\n".format(drive_name, json.dumps(tokens)))
        if drive_type == "Google Shared Drive":
            wfile.write("team_drive = {}".format(root_id))
        else:
            wfile.write("root_folder_id = {}".format(root_id))
        wfile.close()

        print("\nConfiguration has been appended to rclone.conf.")
    else:
        print("Failed to retrieve tokens")
        print("Status Code:", response.status_code)
        print("Response:", response.json())

# Copy from Drive 1 to Drive 2

In [ ]:
# @markdown Enter Source File/Folder in Drive1. Note that in case of folder, the files and subfolders inside are copied. Leave blank for root.
source = "Documents/ABC.pdf" #@param {type:"string"}
# @markdown Enter Destination Folder in Drive2. Leave blank for root
dest = "MyDocs" #@param {type:"string"}


# Construct the full source and destination paths
if source:
  source_path = f"Drive1:/{source}"
else:
  source_path = "Drive1:"

if dest:
  dest_path = f"Drive2:/{dest}"
else:
  dest_path = "Drive2:"

# Copy from Drive 1 to Drive 2
!rclone copy "{source_path}" "{dest_path}" --server-side-across-configs -vv

# About Shared Files/Folders

## Google Drive:
To add a shared file or folder to your Google Drive, follow these steps:
1. Add the shared file or folder as a shortcut to your MyDrive root.
2. Once added, the file or folder will function like any other normal file or folder in your Google Drive.

## OneDrive:
Currently, OneDrive supports shared files and folders in the following categories:
- Personal to Personal Shares
- Shares within the same organization

If you have a shared file or folder that falls under these categories, follow these steps:
1. Go to OneDrive shared section.
2. Make a shortcut of the shared file or folder in your OneDrive root.
3. Once added, the file or folder will function like any other normal file or folder in your OneDrive.